<a href="https://colab.research.google.com/github/fongbubble/UoB_EFIMM0142_Group12_NBA/blob/main/CEC_NBA_wo_schedule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$\text{Supanu Tanatammatid 2546980}$$

### Package

In [1]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils

In [2]:
from pulp import GLPK
import pandas as pd # Library to handle data
import numpy as np
import math
from tabulate import tabulate
from statistics import mean

In [3]:
url = "https://raw.githubusercontent.com/fongbubble/EFIMM0142_NBA_CEC_DEA/main/nba_updated_withrest1.csv"
data = pd.read_csv(url)
data = data.rename(columns={
    "Head Coach's Experience": "Head_Coachs_Experience",
    "Head Coach's Ability": "Head_Coachs_Ability",
    "No. of Games with Rest ": "No_of_Games_with_Rest"
})

### CCR


In [4]:
weights_matrix = np.empty((16, 9))
objective_values = np.empty(16)
for k in range(9):
    for i in range(16):
        model = pulp.LpProblem("NBA", pulp.LpMaximize)  # Create an LP maximization problem

        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget
        v3 = pulp.LpVariable("v3", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Experience
        v4 = pulp.LpVariable("v4", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Ability
        v5 = pulp.LpVariable("v5", lowBound=0, upBound=None, cat='Continuous')  # Roster average age
        v6 = pulp.LpVariable("v6", lowBound=0, upBound=None, cat='Continuous')  # Average years of experience in the league

        # Objective function
        model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
        )

        # First constraint (equality)
        model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] +
            v3 * data["Head_Coachs_Experience"].iloc[i] +
            v4 * data["Head_Coachs_Ability"].iloc[i] +
            v5 * data["Roster average age"].iloc[i] +
            v6 * data["Average years of experience in the league"].iloc[i] == 1
        )

        # Additional constraints
        for j in range(16):
            model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j] -
                v1 * data["AFTbudget"].iloc[j] -
                v2 * data["ABTBudget"].iloc[j] -
                v3 * data["Head_Coachs_Experience"].iloc[j] -
                v4 * data["Head_Coachs_Ability"].iloc[j] -
                v5 * data["Roster average age"].iloc[j] -
                v6 * data["Average years of experience in the league"].iloc[j] <= 0
            )

        # Solve the problem
        status = model.solve(pulp.GLPK(msg=True, options=['--ranges', 'sensitivity_wo_schedule.txt']))
        list_weights = []
        for v in model.variables():
            list_weights.append(v.varValue)
        weights_matrix[i, 0] = list_weights[0]  # m1 -> Column 0 (Scenario_1)
        weights_matrix[i, 1] = list_weights[1]  # m2 -> Column 1 (Scenario_2)
        weights_matrix[i, 2] = list_weights[2]  # m3 -> Column 2 (Scenario_3)
        weights_matrix[i, 3] = list_weights[3]  # n1 -> Column 3 (Scenario_4)
        weights_matrix[i, 4] = list_weights[4]  # n2 -> Column 4 (Scenario_5)
        weights_matrix[i, 5] = list_weights[5]  # n3 -> Column 5 (Scenario_6)
        weights_matrix[i, 6] = list_weights[6]  # n4 -> Column 6 (Scenario_7)
        weights_matrix[i, 7] = list_weights[7]  # n5 -> Column 7 (Scenario_8)
        weights_matrix[i, 8] = list_weights[8]  # n6 -> Column 8 (Scenario_9)

        objective_value = pulp.value(model.objective)
        objective_values[i] = objective_value

team_names = data['Team'].tolist()
scenario_names = [f'Scenario_{k+1}' for k in range(9)]
weights_df = pd.DataFrame(weights_matrix, index=team_names, columns=scenario_names)
objective_values_array = objective_values.reshape(16, 1)
print("Objective Values Array:")
print(objective_values_array)


Objective Values Array:
[[1.00000028]
 [1.0000023 ]
 [0.999999  ]
 [1.00000177]
 [0.8747571 ]
 [1.        ]
 [0.851205  ]
 [0.8053689 ]
 [0.9723903 ]
 [0.8153088 ]
 [0.9450454 ]
 [0.999999  ]
 [0.8311762 ]
 [1.0000002 ]
 [0.8180209 ]
 [0.74014   ]]


### Second Objective

In [5]:
# Calculate New Weights using Second Objective Model
CEC_weights_matrix = np.empty((16, 9))
for k in range(9):
    for i in range(16):
        Second_Objective_model = pulp.LpProblem("NBA_CEC", pulp.LpMaximize)

        u1 = pulp.LpVariable("u1", lowBound=0, upBound=None, cat='Continuous')  # Games Won
        u2 = pulp.LpVariable("u2", lowBound=0, upBound=None, cat='Continuous')  # Playoff -Depth
        u3 = pulp.LpVariable("u3", lowBound=0, upBound=None, cat='Continuous')  # Average Margin of Victory
        v1 = pulp.LpVariable("v1", lowBound=0, upBound=None, cat='Continuous')  # AFTbudget
        v2 = pulp.LpVariable("v2", lowBound=0, upBound=None, cat='Continuous')  # ABTBudget
        v3 = pulp.LpVariable("v3", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Experience
        v4 = pulp.LpVariable("v4", lowBound=0, upBound=None, cat='Continuous')  # Head Coach's Ability
        v5 = pulp.LpVariable("v5", lowBound=0, upBound=None, cat='Continuous')  # Roster average age
        v6 = pulp.LpVariable("v6", lowBound=0, upBound=None, cat='Continuous')  # Average years of experience in the league

        # Define the objective function
        Second_Objective_model += (
            u1 * data["Games Won"].sum() +
            u2 * data["Playoff -Depth"].sum() +
            u3 * data["Average Margin of Victory"].sum()
            - (v1 * data["AFTbudget"].sum() +
            v2 * data["ABTBudget"].sum() +
            v3 * data["Head_Coachs_Experience"].sum() +
            v4 * data["Head_Coachs_Ability"].sum() +
            v5 * data["Roster average age"].sum() +
            v6 * data["Average years of experience in the league"].sum())
        )

        # Define the equality constraint
        Second_Objective_model += (
            v1 * data["AFTbudget"].iloc[i] +
            v2 * data["ABTBudget"].iloc[i] +
            v3 * data["Head_Coachs_Experience"].iloc[i] +
            v4 * data["Head_Coachs_Ability"].iloc[i] +
            v5 * data["Roster average age"].iloc[i] +
            v6 * data["Average years of experience in the league"].iloc[i] == 1
        )

        # Define the inequalities for each team
        for j in range(16):
            Second_Objective_model += (
                u1 * data["Games Won"].iloc[j] +
                u2 * data["Playoff -Depth"].iloc[j] +
                u3 * data["Average Margin of Victory"].iloc[j]
                - (v1 * data["AFTbudget"].iloc[j] +
                   v2 * data["ABTBudget"].iloc[j] +
                   v3 * data["Head_Coachs_Experience"].iloc[j] +
                   v4 * data["Head_Coachs_Ability"].iloc[j] +
                   v5 * data["Roster average age"].iloc[j] +
                   v6 * data["Average years of experience in the league"].iloc[j]) <= 0
            )

        # Additional constraint using the list of objective values
        Second_Objective_model += (
            u1 * data["Games Won"].iloc[i] +
            u2 * data["Playoff -Depth"].iloc[i] +
            u3 * data["Average Margin of Victory"].iloc[i]
            - (objective_values_array[i, 0] * (
                v1 * data["AFTbudget"].iloc[i] +
                v2 * data["ABTBudget"].iloc[i] +
                v3 * data["Head_Coachs_Experience"].iloc[i] +
                v4 * data["Head_Coachs_Ability"].iloc[i] +
                v5 * data["Roster average age"].iloc[i] +
                v6 * data["Average years of experience in the league"].iloc[i])) == 0
        )

        # Solve the model
        status = Second_Objective_model.solve()
        print("Model Status:{}".format(pulp.LpStatus[Second_Objective_model.status]))
        new_list_weights = []
        for v in Second_Objective_model.variables():
            v.varValue = max(0, v.varValue)
            new_list_weights.append(v.varValue)
            print(v.name, "=", v.varValue)
        CEC_weights_matrix[i, 0] = new_list_weights[0]  # u1 -> Column 0 (Scenario_1)
        CEC_weights_matrix[i, 1] = new_list_weights[1]  # u2 -> Column 1 (Scenario_2)
        CEC_weights_matrix[i, 2] = new_list_weights[2]  # u3 -> Column 2 (Scenario_3)
        CEC_weights_matrix[i, 3] = new_list_weights[3]  # v1 -> Column 3 (Scenario_4)
        CEC_weights_matrix[i, 4] = new_list_weights[4]  # v2 -> Column 4 (Scenario_5)
        CEC_weights_matrix[i, 5] = new_list_weights[5]  # v3 -> Column 5 (Scenario_6)
        CEC_weights_matrix[i, 6] = new_list_weights[6]  # v4 -> Column 6 (Scenario_7)
        CEC_weights_matrix[i, 7] = new_list_weights[7]  # v5 -> Column 7 (Scenario_8)
        CEC_weights_matrix[i, 8] = new_list_weights[8]  # v6 -> Column 8 (Scenario_9)
        print("\nConstraint")
        for name, c in Second_Objective_model.constraints.items():
            print(name, ":", c)
CEC_weights_df = pd.DataFrame(CEC_weights_matrix, index=team_names, columns=[f'Scenario_{k+1}' for k in range(9)])
print("Final CEC Weights Matrix with Team Names and Scenario Labels:")
print(CEC_weights_df)


Model Status:Optimal
u1 = 0.013660761
u2 = 0.02514231
u3 = 0
v1 = 0
v2 = 0.038764851
v3 = 0
v4 = 0.9607155
v5 = 0.0073617467
v6 = 0

Constraint
_C1 : 31.3*v1 + 2.55*v2 + 5.1*v3 + 0.738*v4 + 26.1*v5 + 4.7*v6 = 1.0
_C2 : 64*u1 + 5*u2 + 11.3*u3 - 31.3*v1 - 2.55*v2 - 5.1*v3 - 0.738*v4 - 26.1*v5 - 4.7*v6 <= -0.0
_C3 : 57*u1 + 2*u2 + 5.3*u3 - 30.4*v1 - 2.61*v2 - 6.72*v3 - 0.556*v4 - 26.3*v5 - 4.88*v6 <= -0.0
_C4 : 57*u1 + 2*u2 + 7.4*u3 - 14.0*v1 - 5.64*v2 - 5.76*v3 - 0.446*v4 - 24.7*v5 - 3.41*v6 <= -0.0
_C5 : 56*u1 + 3*u2 + 6.5*u3 - 23.8*v1 - 3.51*v2 - 5.66*v3 - 0.537*v4 - 25.6*v5 - 5.1*v6 <= -0.0
_C6 : 51*u1 + u2 + 3.3*u3 - 31.2*v1 - 4.01*v2 - 6.27*v3 - 0.527*v4 - 28.4*v5 - 6.71*v6 <= -0.0
_C7 : 50*u1 + 4*u2 + 2.2*u3 - 23.3*v1 - 4.27*v2 - 6.43*v3 - 0.521*v4 - 25.6*v5 - 4.73*v6 <= -0.0
_C8 : 50*u1 + 2*u2 + 4.6*u3 - 22.0*v1 - 3.98*v2 - 6.82*v3 - 0.574*v4 - 26.4*v5 - 5.0*v6 <= -0.0
_C9 : 49*u1 + u2 + 2.6*u3 - 29.5*v1 - 3.46*v2 - 7.55*v3 - 0.569*v4 - 27.9*v5 - 7.05*v6 <= -0.0
_C10 : 49*u1 + u2 

### Cross-efficiency matrix

In [6]:
CEC_efficiency_table = np.empty((16, 16))
CEC_average_list = []
for team in range(16):
    row_values = []
    for scenario in range(16):
        numerator = (
            (CEC_weights_matrix[scenario, 0] * data['Games Won'].iloc[team]) +
            (CEC_weights_matrix[scenario, 1] * data['Playoff -Depth'].iloc[team]) +
            (CEC_weights_matrix[scenario, 2] * data['Average Margin of Victory'].iloc[team])
        )
        denominator = (
            (CEC_weights_matrix[scenario, 3] * data['AFTbudget'].iloc[team]) +
            (CEC_weights_matrix[scenario, 4] * data['ABTBudget'].iloc[team]) +
            (CEC_weights_matrix[scenario, 5] * data["Head_Coachs_Experience"].iloc[team]) +
            (CEC_weights_matrix[scenario, 6] * data["Head_Coachs_Ability"].iloc[team]) +
            (CEC_weights_matrix[scenario, 7] * data['Roster average age'].iloc[team]) +
            (CEC_weights_matrix[scenario, 8] * data['Average years of experience in the league'].iloc[team])
        )
        if denominator == 0:
            denominator = 1e-10

        # Calculate the efficiency value
        value = numerator / denominator
        CEC_efficiency_table[team, scenario] = value
        row_values.append(value)
    average = np.mean(row_values)
    CEC_average_list.append(average)


In [7]:
CEC_efficiency_df = pd.DataFrame(CEC_efficiency_table, index=team_names, columns=[name + "_weight" for name in team_names])
print("Efficiency Table with Team Names:")
print(CEC_efficiency_df.to_string(formatters={col: '{:.2f}'.format for col in CEC_efficiency_df.columns}))
csv_filename = "2nd_efficiency_table_wo_rest.csv"
CEC_efficiency_df.to_csv(csv_filename, index=True)

Efficiency Table with Team Names:
                       Boston Celtics_weight Denver Nuggets_weight Oklahoma City Thunder_weight Minnesota Timberwolves_weight Los Angeles Clippers_weight Dallas Mavericks_weight New York Knicks_weight Milwaukee Bucks_weight New Orleans Pelicans_weight Phoenix Suns_weight Cleveland Cavaliers_weight Indiana Pacers_weight Los Angeles Lakers_weight Orlando Magic_weight Philadelphia 76ers_weight Miami Heat_weight
Boston Celtics                          1.00                  1.00                         1.00                          1.00                        1.00                    1.00                   1.00                   0.88                        1.00                0.88                       0.96                  1.00                      1.00                 0.68                      1.00              1.00
Denver Nuggets                          1.00                  1.00                         1.00                          1.00                 

### Arithmetic mean for average cross-efficiency score

In [11]:
CEC_team_efficiency = list(zip(data['Team'], CEC_average_list))
sorted_CEC_team_efficiency = sorted(CEC_team_efficiency, key=lambda x: x[1], reverse=True)
for team, efficiency in sorted_CEC_team_efficiency:
    print(f"Average Cross-efficiency score for {team} = {round(efficiency, 3)}")

Average Cross-efficiency score for Oklahoma City Thunder = 1.0
Average Cross-efficiency score for Boston Celtics = 0.963
Average Cross-efficiency score for Minnesota Timberwolves = 0.952
Average Cross-efficiency score for Denver Nuggets = 0.939
Average Cross-efficiency score for Cleveland Cavaliers = 0.869
Average Cross-efficiency score for Indiana Pacers = 0.858
Average Cross-efficiency score for New Orleans Pelicans = 0.85
Average Cross-efficiency score for Dallas Mavericks = 0.834
Average Cross-efficiency score for New York Knicks = 0.797
Average Cross-efficiency score for Los Angeles Clippers = 0.771
Average Cross-efficiency score for Orlando Magic = 0.759
Average Cross-efficiency score for Phoenix Suns = 0.741
Average Cross-efficiency score for Milwaukee Bucks = 0.737
Average Cross-efficiency score for Los Angeles Lakers = 0.735
Average Cross-efficiency score for Philadelphia 76ers = 0.733
Average Cross-efficiency score for Miami Heat = 0.689


### Rescale matrix

In [12]:
tolerance = 1e-6
close_to_one = np.isclose(CEC_efficiency_table, 1.0, atol=tolerance)
count_close_to_one_per_row = np.sum(close_to_one, axis=1)

# Step 1: Find the row with the maximum count of values close to 1.0
row_with_max_ones = np.argmax(count_close_to_one_per_row)

# Step 2: Rescale Matrix
Rescale_Matrix = np.zeros_like(CEC_efficiency_table)
for i in range(CEC_efficiency_table.shape[1]):
    Rescale_Matrix[:, i] = CEC_efficiency_table[:, i] / (CEC_efficiency_table[row_with_max_ones, i] + 1e-10)
Rescale_Matrix = np.round(Rescale_Matrix, 4)

# Step 3: Compute Geometric Mean for Each DMU (Rows)
geometric_mean = [np.prod(Rescale_Matrix[i, :]) ** (1 / Rescale_Matrix.shape[1]) for i in range(Rescale_Matrix.shape[0])]

# Step 4: Calculate Root Sum of Squared Deviations
root_sum_squared_list = []
for col in range(Rescale_Matrix.shape[1]):
    column = Rescale_Matrix[:, col]
    mean = geometric_mean[col]
    deviations = math.sqrt(sum((x - mean) ** 2 for x in column))
    root_sum_squared_list.append(deviations)

# Step 5: Calculate Evaluation Consensus Degree (ECD)
ECD = [1 / (1 + x) for x in root_sum_squared_list]

# Step 6: Calculate Consensus Efficiency Cross (CEC) Results
total_power = sum(ECD)
CEC_Results = []
for i in range(len(ECD)):
    row = Rescale_Matrix[i]
    product = np.prod(np.power(row + 1e-10, ECD[i]))
    result = np.power(product, 1 / total_power)
    CEC_Results.append(result)
CEC_Results = np.array(CEC_Results).round(decimals=4)

# Step 7: Rank All DMUs Based on Final Aggregated Efficiency Scores
df_results = pd.DataFrame({
    'Team': team_names,
    'Consensus Efficiency Score': CEC_Results
})
ECD_df = pd.DataFrame(ECD)
ECD_df.insert(0, 'Team', team_names)
ECD_df.to_csv('ECD_output_wo_rest.csv', index=False)
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)

### Cross-efficiency consensus score

In [13]:
df_results = pd.DataFrame({
    'Team': team_names,
    'Consensus Efficiency Score': CEC_Results
})
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)
for index, row in df_sorted.iterrows():
    print(f"Cross-efficiency consensus score for {row['Team']} = {row['Consensus Efficiency Score']:.4f}")

Cross-efficiency consensus score for Oklahoma City Thunder = 1.0000
Cross-efficiency consensus score for Boston Celtics = 0.9590
Cross-efficiency consensus score for Minnesota Timberwolves = 0.9469
Cross-efficiency consensus score for Denver Nuggets = 0.9275
Cross-efficiency consensus score for New Orleans Pelicans = 0.8639
Cross-efficiency consensus score for Cleveland Cavaliers = 0.8512
Cross-efficiency consensus score for Indiana Pacers = 0.8400
Cross-efficiency consensus score for Dallas Mavericks = 0.8252
Cross-efficiency consensus score for New York Knicks = 0.7830
Cross-efficiency consensus score for Orlando Magic = 0.7747
Cross-efficiency consensus score for Los Angeles Clippers = 0.7594
Cross-efficiency consensus score for Phoenix Suns = 0.7344
Cross-efficiency consensus score for Philadelphia 76ers = 0.7325
Cross-efficiency consensus score for Milwaukee Bucks = 0.7315
Cross-efficiency consensus score for Los Angeles Lakers = 0.7251
Cross-efficiency consensus score for Miami H

### Results comparison

In [10]:
df_sorted = df_results.sort_values(by='Consensus Efficiency Score', ascending=False).reset_index(drop=True)
CEC_average_df = pd.DataFrame(CEC_average_list, columns=['Average Cross-efficiency'], index=team_names)
merged_df = pd.merge(df_sorted, CEC_average_df, left_on='Team', right_index=True)
merged_df = merged_df[['Team', 'Average Cross-efficiency', 'Consensus Efficiency Score']]
merged_df['Average Cross-efficiency'] = merged_df['Average Cross-efficiency'].round(4)
merged_df['Consensus Efficiency Score'] = merged_df['Consensus Efficiency Score'].round(4)
beautiful_table = tabulate(merged_df, headers='keys', tablefmt='pretty', showindex=False)
print(beautiful_table)

+------------------------+--------------------------+----------------------------+
|          Team          | Average Cross-efficiency | Consensus Efficiency Score |
+------------------------+--------------------------+----------------------------+
| Oklahoma City Thunder  |           1.0            |            1.0             |
|     Boston Celtics     |          0.9627          |           0.959            |
| Minnesota Timberwolves |          0.9522          |           0.9469           |
|     Denver Nuggets     |          0.9392          |           0.9275           |
|  New Orleans Pelicans  |          0.8495          |           0.8639           |
|  Cleveland Cavaliers   |          0.8691          |           0.8512           |
|     Indiana Pacers     |          0.8579          |            0.84            |
|    Dallas Mavericks    |          0.8339          |           0.8252           |
|    New York Knicks     |          0.797           |           0.783            |
|   